# Particle database

In [ ]:
# To run in in Google Colab, uncomment the following:

# !pip install expertsystem graphviz

In PWA, you usually want to search for special resonances, possibly even some not listed in the PDG. In this notebook, we go through a few ways to add or overwrite {class}`.Particle` instances in the database with your own particle definitions.

## Loading the default database

In {doc}`reaction`, we made use of the {class}`.StateTransitionManager`. By default, if you do not specify the `particles` argument, the {class}`.StateTransitionManager` calls the function {func}`.load_default_particles`. This functions returns a {class}`.ParticleCollection` instance with {class}`.Particle` definitions from the [PDG](https://pdg.lbl.gov), along with additional definitions that are provided in the file {download}`additional_definitions.yml <../../src/expertsystem/particle/additional_definitions.yml>`.

Here, we call this method directly to illustrate what happens (we use {func}`.load_pdg`, which loads a subset):

In [ ]:
from expertsystem.particle import load_pdg

particle_db = load_pdg()
print("Number of loaded particles:", len(particle_db))

In the following, we illustrate how to use the methods of the {class}`.ParticleCollection` class to find and 'modify' {class}`.Particle`s and {meth}`~.ParticleCollection.add` them back to the {class}`.ParticleCollection`.

## Finding particles

The {class}`.ParticleCollection` class offers some methods to search for particles by name or by PID (see {meth}`~.ParticleCollection.find`):

In [ ]:
particle_db.find(333)

With {meth}`~.ParticleCollection.filter`, you can perform more sophisticated searches. This is done by either passing a function or [lambda](https://docs.python.org/3/tutorial/controlflow.html#lambda-expressions).

In [ ]:
subset = particle_db.filter(lambda p: p.name.startswith("f(2)"))
subset.names

In [ ]:
subset = particle_db.filter(
    lambda p: p.strangeness == 1
    and p.spin >= 1
    and p.mass > 1.8
    and p.mass < 1.9
)
subset.names

In [ ]:
subset = particle_db.filter(lambda p: p.is_lepton())
subset.names

Note that in each of these examples, we call the {attr}`~.ParticleCollection.names` property. This is just to only display the names, sorted alphabetically, otherwise the output becomes a bit of a mess:

In [ ]:
particle_db.filter(lambda p: p.name.startswith("pi") and len(p.name) == 3)

## LaTeX representation

{class}`.Particle`s also contain a {attr}`~.Particle.latex` tag. Here, we use [ipython](https://ipython.readthedocs.io/en/stable/api/generated/IPython.display.html#IPython.display.Math) to render them nicely as mathematical symbols:

In [ ]:
from IPython.display import Math

sigmas = particle_db.filter(
    lambda p: p.name.startswith("Sigma") and p.charmness == 1
)
Math(", ".join([p.latex for p in sigmas]))

## Adding custom particle definitions through Python

A quick way to modify or overwrite particles, is through your Python script or notebook. Notice that the instances in the database are {class}`.Particle` instances:

In [ ]:
N1650_plus = particle_db["N(1650)+"]
N1650_plus

The instances in the database are [immutable](https://en.wikipedia.org/wiki/Immutable_object). Therefore, if you want to modify, say, the width, you have to create a new {class}`.Particle` instance from the particle you want to modify and {meth}`~.ParticleCollection.add` it back to the database. You can do this with {func}`.create_particle`:

```{margin} Duplicate names or PIDs
The warning that you see here comes from the fact that names and PIDs are considered mere labels of a {obj}`.Particle` instance ― it is defined uniquely only by its quantum numbers, such as {attr}`~.Particle.spin` and {attr}`~.Particle.charge`. 
    
The warning is suppressed in the rest of this page.
```

In [ ]:
from expertsystem.particle import create_particle

new_N1650_plus = create_particle(
    template_particle=N1650_plus, name="Modified N(1650)+", width=0.2
)

particle_db.add(new_N1650_plus)
particle_db["Modified N(1650)+"].width

You often also want to add the antiparticle of the particle you modified to the database. Using {func}`.create_antiparticle`, it is easy to create the corresponding antiparticle object.

In [ ]:
from expertsystem.particle import create_antiparticle

new_N1650_minus = create_antiparticle(
    new_N1650_plus, new_name="Modified N(1650)-"
)

particle_db.add(new_N1650_minus)
particle_db["Modified N(1650)-"]

When adding additional particles you may need for your research, it is easiest to work with an existing particle as template. Let's say we want to study $e^+e^-$ collisions of several energies:

````{margin}
```{note}
By convention, the {mod}`expertsystem` uses $\mathrm{GeV}/c^2$ as energy unit.
```
````

In [ ]:
energies_mev = {4180, 4220, 4420, 4600}
template_particle = particle_db["J/psi(1S)"]
for energy_mev in energies_mev:
    energy_gev = energy_mev / 1e3
    new_particle = create_particle(
        template_particle, name=f"EpEm ({energy_mev} MeV)", mass=energy_gev
    )
    particle_db.add(new_particle)
len(particle_db)

In [ ]:
particle_db.filter(lambda p: "EpEm" in p.name).names

## Loading custom definitions from a YAML file

It's also possible to add particles from a config file, with {func}`.io.load`. Existing entries remain and if the imported file of particle definitions contains a particle with the same name, it is overwritten in the database.

It's easiest to work with YAML. Here, we use the provided {download}`additional_particles.yml` example file:

In [ ]:
from expertsystem import io

particle_db += io.load("additional_particles.yml")

## Writing to YAML

You can also dump the existing particle lists to YAML. You do this with the {func}`.io.write` function.

In [ ]:
io.write(instance=particle_db, filename="dumped_particle_list.yaml")

Note that the function {func}`write <.io.write>` can dump any {class}`.ParticleCollection` to an output file, also a specific subset.

In [ ]:
from expertsystem.particle import ParticleCollection

output = ParticleCollection()
output += particle_db["J/psi(1S)"]
output += particle_db.find(22)  # gamma
output += particle_db.filter(lambda p: p.name.startswith("f(0)"))
output += particle_db["pi0"]
output += particle_db["pi+"]
output += particle_db["pi-"]
io.write(output, "particle_list_selection.yml")
output.names

As a side note, the {mod}`expertsystem` provides [JSON schemas](https://json-schema.org) ({download}`particle/validation.json <../../src/expertsystem/particle/validation.json>`) to validate your particle list files (see also {func}`jsonschema.validate`). If you have installed the {mod}`expertsystem` as an {ref}`pwa:develop:Editable installation` and {ref}`use VSCode <develop:Visual Studio code>`, your YAML particle list are checked automatically in the GUI.